In [1]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import  PCA
from sklearn.metrics import roc_auc_score

import numpy as np

In [2]:
num_test = 1000

label=['signal','lepton 1 pT', 'lepton 1 eta', 'lepton 1 phi', 'lepton 2 pT', 'lepton 2 eta', 'lepton 2 phi', 'missing energy magnitude', 'missing energy phi', 'MET_rel', 'axial MET', 'M_R', 'M_TR_2', 'R', 'MT2', 'S_R', 'M_Delta_R', 'dPhi_r_b', 'cos(theta_r1)']
var=['lepton 1 pT','lepton 2 pT','missing energy magnitude']

df=pd.read_csv('~/SageMaker/SUSY_cut.csv',nrows=(num_test))



Xdata=df[var].to_numpy()
Ydata=df['signal'].to_numpy()

In [3]:
min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(Xdata)

X_test=X_scaled[:num_test]
y_test=Ydata[:num_test]

In [8]:
def decimalToBinary(n, length):
    binary = bin(n).replace("0b", "")
    if len(binary) != length:
        for i in range(length - len(binary)):
            binary = "0" + binary

    return binary


def get_min_fidel(andrei_prob):

    andrei_qubit = int(np.log2(len(andrei_prob)))
    bit_length = andrei_qubit
    qubit_prob = np.zeros((andrei_qubit, 2))


    for i in range(len(andrei_prob)):
        bit = decimalToBinary(i, bit_length)

        for j in range(len(bit)):
            qubit_prob[j, int(bit[j])] += andrei_prob[i]
    return qubit_prob



In [24]:
fidel=[]
for i in range(100):
    
    pron=np.loadtxt('Probability_Smart_Runn/andrei_prob_'+str(i)+'.txt')
    min_fid=get_min_fidel(pron)
    for j in min_fid:
        fidel.append(j)
print('len_fid',len(fidel))

len_fid 1000


In [29]:
def test(fidel, y, state_labels=None):
    """
    Tests on a given set of data.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 3-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        predicted (array([int]): predicted labels for test data
        output_states (array[float]): output quantum states from the circuit
    """
    predicted = []
    fidelity_values=fidel
    for i in fidel:
        
        best_fidel = np.argmax(i)

        predicted.append(best_fidel)
        

    return np.array(predicted), np.array(fidelity_values)

def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    score = y_true == y_pred
    return score.sum() / len(y_true)

In [30]:
predicted_test, fidel_test = test(fidel, y_test)
accuracy_test = accuracy_score(y_test, predicted_test)
AUC=roc_auc_score(y_test,fidel_test[:,1])
print("Accuracy:{}|AUC:{}. ".format(accuracy_test,AUC))

Accuracy:0.572|AUC:0.7780906262768466. 
